In [6]:
import os
os.environ["OPENAI_API_KEY"]=<Refer>

In [58]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(temperature=1,model='gpt-4o-2024-05-13')

In [66]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field,TypeAdapter

class Player(BaseModel):
    
    name: str = Field(description="Name of the player")
    age: str = Field(description="Age of the player")
    specialization: str = Field(description="Sepcialization of the player")
    origin:str = Field(description= "Origin of the player")


class Players(BaseModel):
    players: list = Field(description="Python list of Player")

parser = PydanticOutputParser(pydantic_object=Players)

In [33]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"players": {"description": "Python list of Player", "items": {}, "title": "Players", "type": "array"}}, "required": ["players"]}\n```'

In [78]:
from langchain_core.prompts import ChatPromptTemplate

def getChatPrompt(field):
    chatpromt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a {field} Analyst"
        ),
        (
            "human"
            "Get me the name of all the current Playing Indian Players name, their specialization, their age and origin \n {format_instructions}"
        )
    ]
    )

    chatpromtvalue = chatpromt.invoke(
    {
        "field":"cricket",
        "format_instructions":parser.get_format_instructions()
    }
    ) 
    return 





chatpromtvalue


ChatPromptValue(messages=[SystemMessage(content='You are a cricket Analyst'), HumanMessage(content='humanGet me the name of all the current Playing Indian Players name, their specialization, their age and origin \n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"players": {"description": "Python list of Player", "items": {}, "title": "Players", "type": "array"}}, "required": ["players"]}\n```')])

In [79]:
result =  model.invoke(chatpromtvalue)
result

AIMessage(content='```json\n{\n  "players": [\n    {\n      "name": "Rohit Sharma",\n      "specialization": "Batsman",\n      "age": 36,\n      "origin": "Nagpur, Maharashtra"\n    },\n    {\n      "name": "Virat Kohli",\n      "specialization": "Batsman",\n      "age": 34,\n      "origin": "Delhi"\n    },\n    {\n      "name": "KL Rahul",\n      "specialization": "Batsman/Wicket-keeper",\n      "age": 31,\n      "origin": "Mangalore, Karnataka"\n    },\n    {\n      "name": "Jasprit Bumrah",\n      "specialization": "Bowler",\n      "age": 29,\n      "origin": "Ahmedabad, Gujarat"\n    },\n    {\n      "name": "Ravindra Jadeja",\n      "specialization": "All-rounder",\n      "age": 34,\n      "origin": "Navagam, Gujarat"\n    },\n    {\n      "name": "Ravichandran Ashwin",\n      "specialization": "Bowler",\n      "age": 37,\n      "origin": "Chennai, Tamil Nadu"\n    },\n    {\n      "name": "Hardik Pandya",\n      "specialization": "All-rounder",\n      "age": 30,\n      "origin": 

In [80]:
players = parser.parse(result.content)
players.players

[{'name': 'Rohit Sharma',
  'specialization': 'Batsman',
  'age': 36,
  'origin': 'Nagpur, Maharashtra'},
 {'name': 'Virat Kohli',
  'specialization': 'Batsman',
  'age': 34,
  'origin': 'Delhi'},
 {'name': 'KL Rahul',
  'specialization': 'Batsman/Wicket-keeper',
  'age': 31,
  'origin': 'Mangalore, Karnataka'},
 {'name': 'Jasprit Bumrah',
  'specialization': 'Bowler',
  'age': 29,
  'origin': 'Ahmedabad, Gujarat'},
 {'name': 'Ravindra Jadeja',
  'specialization': 'All-rounder',
  'age': 34,
  'origin': 'Navagam, Gujarat'},
 {'name': 'Ravichandran Ashwin',
  'specialization': 'Bowler',
  'age': 37,
  'origin': 'Chennai, Tamil Nadu'},
 {'name': 'Hardik Pandya',
  'specialization': 'All-rounder',
  'age': 30,
  'origin': 'Surat, Gujarat'},
 {'name': 'Shreyas Iyer',
  'specialization': 'Batsman',
  'age': 28,
  'origin': 'Mumbai, Maharashtra'},
 {'name': 'Rishabh Pant',
  'specialization': 'Batsman/Wicket-keeper',
  'age': 26,
  'origin': 'Roorkee, Uttarakhand'},
 {'name': 'Mohammed Shami